In [1]:
%pip install git+https://github.com/iuts-beb-chargers/2025-CET-593-beb_chargers.git
%pip install -r https://raw.githubusercontent.com/iuts-beb-chargers/2025-CET-593-beb_chargers/main/requirements.txt


  Cloning https://github.com/iuts-beb-chargers/2025-CET-593-beb_chargers.git to c:\users\nick_james\appdata\local\temp\pip-req-build-st1cuhiq
  Resolved https://github.com/iuts-beb-chargers/2025-CET-593-beb_chargers.git to commit 72d506adeaed3f6cc3a9277308e1168f6adb87aa
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/iuts-beb-chargers/2025-CET-593-beb_chargers.git 'C:\Users\Nick_James\AppData\Local\Temp\pip-req-build-st1cuhiq'


Note: you may need to restart the kernel to use updated packages.


In [2]:
from beb_chargers.scripts.script_helpers import build_trips_df, \
    build_charger_location_inputs
from beb_chargers.opt.charger_location import ChargerLocationModel
from beb_chargers.gtfs_beb import GTFSData
from beb_chargers.vis import plot_trips_and_terminals, plot_deadhead
from pathlib import Path
import datetime
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# Locate CSV file giving candidate charger sites for this instance
site_fname = r"E:/UW-Seattle/UW/25WI/CET 593/Project1/data/chargers/chargers_nyc.xlsx"

# Directory to GTFS files, as a platform-agnostic path
gtfs_dir = r"E:/UW-Seattle/UW/25WI/CET 593/Project1/data/GTFS"
# Directory to osm files
osm = (r"E:/UW-Seattle/UW/25WI/CET 593/Project1/osm_charge_data.pickle")

In [4]:
# Power output of each charger. Note that the model expects a unit of
# kilowatt-hours per minute, so that the amount of energy gained equals
# charging time (in minutes) multiplied by power (in kWh/min). Here, we
chg_pwrs = 450 / 60

# Maximum number of chargers per site. We'll set it to 4 everywhere
n_max = 4

# Cost parameters. See TRC paper for where the values come from.
s_cost = 500000
c_cost = 700000
alpha = 190 * 365 * 12 / 60

# Load candidate charging sites given by Metro and add params as columns
loc_df = pd.read_excel(site_fname)
loc_df['max_chargers'] = n_max
loc_df['kw'] = chg_pwrs * 60
loc_df['fixed_cost'] = s_cost
loc_df['charger_cost'] = c_cost

# Define coordinates of overnight depot at South Base
depot_coords = (40.76050360027786, -73.99942123020429)

In [5]:
# Load GTFS data into our custom object
gtfs = GTFSData.from_dir(gtfs_dir)

In [6]:
# Battery capacity in kWh
battery_cap = 300
# Energy consumption per mile for all buses (we'll assume it's the same
# for all buses here)
kwh_per_mi = 3

In [7]:
# We'll run analysis for March 28, 2024
ocl_date = datetime.datetime(2024, 3, 28)

In [8]:
import csv
import random

def read_routes_file(filename):
    """
    read routes file and return route_ids list
    Args:
        filename: CSV path
    Returns:
        route_ids list
    """
    route_ids = []
    with open(filename, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            route_ids.append(row['route_id'])
    return route_ids

def random_select_routes(route_ids, percentage=0.1):
    """
    Args:
        route_ids: all routes ID
        percentage
    Returns:
        selected routes ID
    """
    num_to_select = round(len(route_ids) * percentage)
    available_routes = route_ids.copy()
    selected_routes = []
    
    # 随机选择路线
    for _ in range(num_to_select):
        if not available_routes:
            break
        # 随机选择一个索引
        idx = random.randrange(len(available_routes))
        # 添加选中的路线ID并从可选列表中移除
        selected_routes.append(available_routes.pop(idx))
    
    return selected_routes

def main():
    filename = 'routes.txt'
    num_iterations = 10
    
    route_ids = read_routes_file(filename)
    print(f"Total number of routes: {len(route_ids)}")
    
    all_results = []
    
    for i in range(num_iterations):
        selected = random_select_routes(route_ids)
        all_results.append(selected)
        print(f"\nIteration {i+1}:")
        print(f"Selected {len(selected)} routes: {selected}")
    
    return all_results

if __name__ == "__main__":
    random.seed(42)
    
    # 运行主程序
    results = main()
    
    print("\nFinal 2D array of selected route_ids:")
    for i, row in enumerate(results):
        print(f"Row {i+1}: {row}")

Total number of routes: 314

Iteration 1:
Selected 31 routes: ['B92', 'B26', 'M14A+', 'M7', 'FBAS', 'BX15', 'B84', 'Q107', 'B69', 'B106', 'SIM23', 'B36', 'B35', 'B82+', 'KBAS', 'M10', 'X68', 'B31', 'B99', 'GHAS', 'S96B1', 'SIM32', 'M3', 'S59', 'M60+', 'B13', 'BX31', 'S44', 'Q55', 'M79+', 'BX29']

Iteration 2:
Selected 31 routes: ['GHAS', 'Q15', 'B83', 'B70', 'Q77', 'B82', 'Q43', 'Q27', 'M12', 'B41', 'S56', 'Q95', 'BX1', 'Q88', 'B64', 'S96B1', 'M60+', 'Q76', 'B111', 'BX99', 'B6', 'B43', 'M5', 'M79+', 'B68', 'M100', 'BX4', 'SIM8', 'M57', 'S89', 'SIM3C']

Iteration 3:
Selected 31 routes: ['BX22', 'Q46', 'Q28', 'SIR5', 'M116', 'B6', 'BX29', 'M90', 'M9', 'BX25', 'S56', 'Q85', 'M15+', 'S96B2', 'GAAS', 'Q16', 'B46', 'M3', 'B35', 'Q15', 'SIM15', 'M23+', 'B57', 'FBAS', 'Q20B', 'JGAS', 'L90', 'SIM24', 'S90', 'BX18B', 'M34A+']

Iteration 4:
Selected 31 routes: ['BX12+', 'M7', 'B98', 'D90', 'M106', 'B99', 'SIM15', 'B101', 'SIM1C', 'Q43', 'OFAS', 'BX12', 'X37', 'S91', 'B7', 'B43', 'BX3', 'BX22', 'B

In [9]:
all_beb_routes = []

for group_routes in results:
    # Convert each route ID in the current group to string
    beb_routes = [str(route) for route in group_routes]
    all_beb_routes.append(beb_routes)

In [ ]:
import datetime
import multiprocessing
import logging
import os

# 统一日期（所有数据都使用 2024-03-28）
ocl_date = datetime.datetime(2024, 3, 28)

# `all_beb_routes` 是存储 10 组 BEB 线路的列表，每个元素是一组线路
# all_beb_routes = [...]  # 你的 BEB 线路数据

osm_fnames = [osm for _ in range(10)]  # 每组数据对应的 OSM 文件
battery_caps = [battery_cap for _ in range(10)]  # 电池容量保持一致

# 指定存储路径（确保该目录存在）
output_dir = "E:/UW-Seattle/UW/25WI/CET 593/Project1/"  # 修改为你的实际路径
os.makedirs(output_dir, exist_ok=True)  # 如果目录不存在，则创建

def process_data(beb_routes, osm_fname, battery_cap, index):
    """
    并行计算单组数据的充电站优化结果（日期固定为 2024-03-28）
    并将结果保存到 txt 文件
    """
    # 生成 BEB 线路数据
    beb_trips = build_trips_df(
        gtfs=gtfs, date=ocl_date, routes=beb_routes, 
        depot_coords=depot_coords, add_depot_dh=True, 
        add_kwh_per_mi=False, add_durations=False, 
        routes_60=[], osm_fname=osm_fname
    )
    
    # 计算能耗
    beb_trips['kwh_per_mi'] = kwh_per_mi

    # 记录该天 BEB 运行情况
    logging.info('{}: There are {} total trips to be served by {} BEB blocks.'.format(
        ocl_date.strftime('%m/%d/%y'), len(beb_trips), beb_trips['block_id'].nunique()
    ))

    # 构建优化输入
    opt_kwargs = build_charger_location_inputs(
        gtfs=gtfs, trips_df=beb_trips, chargers_df=loc_df, 
        depot_coords=depot_coords, battery_cap=battery_cap, osm_fname=osm_fname
    )

    # 运行充电站优化模型
    clm = ChargerLocationModel(**opt_kwargs)
    clm.solve(alpha=alpha, opt_gap=0, bu_kwh=battery_cap, jupyter_case=osm_fname)

    # 记录优化结果
    clm.log_results()

    # 获取优化结果 DataFrame
    result_df = clm.to_df()

    # 保存结果到 txt 文件
    output_file = os.path.join(output_dir, f"result_group_{index+1}.txt")
    with open(output_file, "w") as f:
        f.write(result_df.to_string(index=False))  # 保存为可读格式，不含索引

    return result_df

# 使用 multiprocessing 进行并行计算，并保存结果到不同的 txt 文件
if __name__ == '__main__':
    # 使用 multiprocessing 并行计算，每组数据对应不同的 BEB 线路
    with multiprocessing.Pool(processes=10) as pool:
        results = pool.starmap(process_data, zip(all_beb_routes, osm_fnames, battery_caps, range(10)))

    # 打印每组数据的前 5 行结果
    for i, df in enumerate(results):
        print(f"Dataset {i+1} results (saved in result_group_{i+1}.txt):")
        display(df.head(5))